# Read Me: 스크린 타입 크롤링 후 분류해주기

- 동일한 영화라도, 3D, 2D, IMax 등등 다양한 영화타입으로 상영되기 때문에, 이런 스크린 타입(일반관,특수관)이 관람객 수에 영향을 미치는지 보고자 함.
----
- screentype_normal = ['필름','디지털','디지털 영문자막', '디지털 가치봄','디지털 더빙','흑백판'] 으로 상영되었던 것은 일반관처리
- screentype_special = ['4D','IMAX','3D 디지털','IMAX 3D','4D 더빙','3D 더빙', 'DOLBYCINEMA', 'ScreenX']으로 상영되었던 것은 특수관처리
----
- Selenium을 이용하였음

# 1. 필요한 모듈

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager    # 매 번 크롬 드라이버를 설치할 필요없이 자동으로
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert
from tqdm.notebook import tqdm
from urllib.request import urlopen
import time
import requests
import os
import shutil

예시
<img src="screentype.png">

In [6]:
driver = webdriver.Chrome("chromedriver")                    
driver.get("https://www.kobis.or.kr/kobis/business/stat/boxs/findFormerBoxOfficeList.do")   # 사이트 접속

# 상영 타입 정리 - '일반관'과 '특별관'
screentype_normal = ['필름','디지털','디지털 영문자막', '디지털 가치봄','디지털 더빙','흑백판']
screentype_special = ['4D','IMAX','3D 디지털','IMAX 3D','4D 더빙','3D 더빙', 'DOLBYCINEMA', 'ScreenX']

normal_list = []                 # 일반관 관람객 수를 담을 리스트
special_list = []                # 특수관 관람객 수를 담을 리스트

for i in tqdm(range(0,56)):
    driver.find_element(By.ID,"tr_{}".format(i)).find_element(By.CLASS_NAME, "ellip").click()              # 영화 클릭
    
    click_xpath = '/html/body/div[2]/div[1]/div[2]/ul/li[2]/a'                                       
     
    element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, click_xpath)))         # 통계 정보 클릭
    
    element.click()             
    
    loading_xpath = '/html/body/div[2]/div[2]/div/div[2]/p/img'                                            # 로딩 기다려주는 시간
    WebDriverWait(driver, 100).until(EC.invisibility_of_element_located((By.XPATH, loading_xpath)))
     
    
    # 상영타입별 누적통계 표 읽어오기
    table = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div/div[2]/div[6]/table/tbody')
    table_rows = table.find_elements(By.TAG_NAME, 'tr')
    
    
    # 일반타입의 상영관 관람객수와 특수관타입의 관람객수를 더해주기 위해 0매번 0으로 초기화 해주어야 함
    normal_sum = 0
    special_sum = 0
    
    for row in table_rows:                                        
        cells = row.find_elements(By.TAG_NAME, 'td')

        first_cell_text = cells[0].text                           # cells[0]은  상영타입(ex.필름, 디지털, 3D, 4D)
        last_cell_text = cells[-1].text                           # cells[-1]은 누적관객수(점유율) (ex. 17,552,467(99.6%) )
    
        
        # 관람객 수만 구하려면, str형이기 때문에 콤마를 없애주고, (를 기준으로 split하여, int형으로 전환해주어야 함.
    
        # 만약, 상영타입이 ['필름','디지털','디지털 영문자막', '디지털 가치봄','디지털 더빙','흑백판'] 중 하나이면 일반관임.
        if first_cell_text in screentype_normal:
            normal_sum += int(last_cell_text.replace(',','').split('(')[0])      # 일반관 관람객 수의 합계

        # 만약, 상영타입이 ['4D','IMAX','3D 디지털','IMAX 3D','4D 더빙','3D 더빙', 'DOLBYCINEMA', 'ScreenX']중 하나이면 특수관임
        else:
            special_sum += int(last_cell_text.replace(',','').split('(')[0])     # 특수관 관람객 수의 합계
        
    
    normal_list.append(normal_sum)                 
    special_list.append(special_sum)               
    
    driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[1]/a[2]').click()        # 영화정보창 닫기
    
print(normal_list)
print(special_list)

  0%|          | 0/58 [00:00<?, ?it/s]

[17563177, 16266338, 14213274, 14264476, 12773929, 12806843, 13413821, 6921137, 12983942, 12810975, 11115370, 12671839, 12627228, 12321693, 11863931, 12189518, 11397668, 11373245, 11284421, 10124019, 10917400, 5072801, 10514177, 8590741, 9117121, 10312318, 7918023, 8873156, 9707581, 9409128, 9353799, 9135729, 7567164, 7869922, 8657366, 8659727, 8252952, 8166508, 6940385, 8007667, 7709460, 7351195, 7802947, 3737420, 7759761, 6829031, 7153713, 7425255, 7425276, 7367394, 7255764, 7363142, 6795554, 6865440, 7232452, 7151476, 7150586, 7120780]
[52742, 0, 201384, 0, 1203673, 943825, 663, 6417726, 759, 1169, 1682557, 35108, 66187, 2365, 414079, 188, 170147, 2154, 40696, 1109157, 0, 5729756, 0, 1913746, 1225402, 1201, 2387028, 1075230, 0, 17832, 0, 77, 1434515, 808839, 8842, 1074, 475, 56834, 1255357, 27514, 314146, 504279, 14739, 4047799, 0, 722959, 422272, 75202, 45357, 35338, 137679, 0, 469380, 393238, 0, 15212, 0, 0]


## 영화 제목 뽑아오기

In [89]:
driver = webdriver.Chrome("chromedriver")                    
driver.get("https://www.kobis.or.kr/kobis/business/stat/boxs/findFormerBoxOfficeList.do")

title = []

for i in driver.find_elements(By.ID, "td_movie"):
    title.append(i.text)

## 1위~56위까지 일반관, 특수관 관람객 수 구하기

In [143]:
result = pd.DataFrame({'Title':title[:56], "일반관":normal_list, "특수관":special_list})  

In [145]:
result['Total'] = result['일반관'] + result['특수관']

In [146]:
result

,Title,일반관,특수관,Total
0,명량,17563177,52742,17615919
1,극한직업,16266338,0,16266338
2,신과함께-죄와 벌,14213274,201384,14414658
3,국제시장,14264476,0,14264476
4,어벤져스: 엔드게임,12773929,1203673,13977602
5,겨울왕국 2,12806843,943825,13750668
6,베테랑,13413821,663,13414484
7,아바타,6921137,6417726,13338863
8,도둑들,12983942,759,12984701
9,7번방의 선물,12810975,1169,12812144


## 실미도와 태극기 휘날리며는 옛날 데이터이기 때문에 따로 추가해주기

In [147]:
# 추가할 행 생성
new_row = pd.DataFrame({
    'Title': ['태극기 휘날리며'],
    '일반관': [11746135],
    '특수관': [0],
    'Total':[11746135]
})

# 20행에 행 추가
result = pd.concat([result.iloc[:16], new_row, result.iloc[16:]], ignore_index=True)

In [148]:
# 추가할 행 생성
new_row = pd.DataFrame({
    'Title': ['실미도'],
    '일반관': [11081000],
    '특수관': [0],
    'Total':[11081000]
})

# 20행에 행 추가
result = pd.concat([result.iloc[:21], new_row, result.iloc[21:]], ignore_index=True)

In [149]:
result

,Title,일반관,특수관,Total
0,명량,17563177,52742,17615919
1,극한직업,16266338,0,16266338
2,신과함께-죄와 벌,14213274,201384,14414658
3,국제시장,14264476,0,14264476
4,어벤져스: 엔드게임,12773929,1203673,13977602
5,겨울왕국 2,12806843,943825,13750668
6,베테랑,13413821,663,13414484
7,아바타,6921137,6417726,13338863
8,도둑들,12983942,759,12984701
9,7번방의 선물,12810975,1169,12812144


In [150]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   58 non-null     object
 1   일반관     58 non-null     int64 
 2   특수관     58 non-null     int64 
 3   Total   58 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.9+ KB


In [151]:
result.to_csv("screentype.csv", encoding="utf-8", index=False)

In [3]:
result = pd.read_csv("screentype.csv")

In [5]:
result.columns = ["영화","일반관","특수관","누적관객수"]

In [7]:
result['일반관(비율)'] = result['일반관'] / result['누적관객수']

In [8]:
result['특수관(비율)'] = result['특수관'] / result['누적관객수']

In [9]:
result

,영화,일반관,특수관,누적관객수,일반관(비율),특수관(비율)
0,명량,17563177,52742,17615919,0.997006,0.002994
1,극한직업,16266338,0,16266338,1.000000,0.000000
2,신과함께-죄와 벌,14213274,201384,14414658,0.986029,0.013971
3,국제시장,14264476,0,14264476,1.000000,0.000000
4,어벤져스: 엔드게임,12773929,1203673,13977602,0.913886,0.086114
5,겨울왕국 2,12806843,943825,13750668,0.931362,0.068638
6,베테랑,13413821,663,13414484,0.999951,0.000049
7,아바타,6921137,6417726,13338863,0.518870,0.481130
8,도둑들,12983942,759,12984701,0.999942,0.000058
9,7번방의 선물,12810975,1169,12812144,0.999909,0.000091
